In [1]:
%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import pandas as pd

#import distance
import nltk
#nltk.download('stopwords')

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

from openpyxl import load_workbook

In [16]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    '2019-01-30-11-22-25_1' : [4, 6, 7], # ys, session 3
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    '2019-01-14-14-58-30' : [0], # ys, session ()
    '2019-01-16-16-36-17_1stPart_2' : [-1],
    '2019-01-16-17-00-12_2ndPart_2': [1],
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session1
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-14-13-28-20_1stPart_2' : [2], # ch_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # ch_session3_2_part2
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    #'2019-02-18-10-28-35_2' : [0],               # ls2_session4
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-02-19-17-10-45_1' : [3]                  # ph_session5
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12' : 'no gaze data',
    '2019-01-17-15-31-12_2' : 'no gaze data'
}



In [ ]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

# normally, reading part of trial ends when people look at the keyboardwithphrases. For some trials, this is not done,
# as the reading is done, and the trial is accidentally skipped, and written in the next trial. Here, the trial number 
# given will have the reading time ending as sleep, and not keyboard with phrases. 
dict_keyboardNotChange_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : 0, # ys_session2 
}

dict_keySelection_firstSleepNotCounted = {
    "2019-01-28-14-50-41_2" : (0, 2), # bh1_session1 -- 3rd sleep activation to be counted
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [4]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [5]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [6]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_dwellTimeOrig_not800:
        TimeDwellOrig = dict_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [7]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [8]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [9]:
def stimPhrasesEdit(PhraseLog, full_path):
   
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
        
    phraseStim_PhrasesReduced, phraseStim_timeReduced = zip(*[(x[0], PhraseLog[phraseStim_Phrases.index(x[0])][0]) for x in groupby(phraseStim_Phrases)])
    
    PhraseLogReduced = [[phraseStim_timeReduced[i], phraseStim_PhrasesReduced[i]] for i in range(0, len(phraseStim_PhrasesReduced))]
    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if PhraseLogReduced[0][1] == 'phraseText':
        del PhraseLogReduced[0]

    # Here, we want only the sentences typed
    notSentencesToType = list()
    for index in range(0,len(PhraseLogReduced)):
        sentence = PhraseLogReduced[index][1]
        if 'Svar på følgende spørgsmål' in sentence or 'Answer the question:' in sentence or 'What is the complete name of your university?' in sentence or '(give a score between 1 and 7)' in sentence or sentence == '':
            notSentencesToType.append(index)
         
    
    for index in sorted(notSentencesToType, reverse=True):
        del PhraseLogReduced[index]
    
    if session_folder_name in dict_phraseStim:
        #print('session in stim phrases found')
        index_to_be_removed = dict_phraseStim[session_folder_name]
    else:
        index_to_be_removed = []
        
    #print(index_to_be_removed)
    
    if index_to_be_removed:
        for index in sorted(index_to_be_removed, reverse=True):
            del PhraseLogReduced[index]
        
    return PhraseLogReduced

In [10]:
def scratchPadPhraseEdit(phraseUser, subjName, full_path, picture):
    phraseUserEnd = list()
    
    
    
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            if phraseUser[row_ind-1][1] != 'scratchPadText':
                if len(phraseUser[row_ind-1][1])>2: # this also removes any answers on the difficulty of the sentence
                    phraseUserEnd.append(phraseUser[row_ind-1])
                    #print(phraseUser[row_ind-1])
                    
    #print(' (1) phrases reduced to :       ', phraseUserEnd)
    # remove first two trials of baseline question and text composition
    if 'not_described' in picture or '2019-01-14-14-58-30' in full_path or '2019-02-06-12-37-45_2' in full_path or '2019-02-18-10-28-35_2' in full_path: 
        # yss has not copied one of the sentences, aq_session1_2 has not written the baseline question, ls2_session4_2 did not describe the picture
        del phraseUserEnd[0]
    else:
        del phraseUserEnd[0:2]
      
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_phraseUser:
        index_to_be_removed = dict_phraseUser[session_folder_name]
        #print('session in user phrases found')
    else:
        index_to_be_removed = []
        
    
    if index_to_be_removed:
        for index in sorted(index_to_be_removed, reverse=True):
            del phraseUserEnd[index]
        
    
    #print(' (2) phrases reduced to:       ', phraseUserEnd)
    
    # remove the initial rating of complexity, if they have written it:
    for index in range(0,len(phraseUserEnd)):
        sentence = phraseUserEnd[index][1]
        if sentence[0].isdigit():
            
            # if there is also a space after the digit:
            
            if sentence[1] == ' ':
                phraseUserEnd[index][1] = phraseUserEnd[index][1][2:]
            else:
                phraseUserEnd[index][1] = phraseUserEnd[index][1][1:]
            
        elif sentence[1].isdigit():
            if sentence[2] == ' ':
                phraseUserEnd[index][1] = phraseUserEnd[index][1][3:]
            else:
                phraseUserEnd[index][1] = phraseUserEnd[index][1][2:]
    
    #print(' (3) phrases reduced to:       ', phraseUserEnd)
    
    return phraseUserEnd

In [11]:
def ComputeCorrectLetters(PhraseStim, PhraseUserEnd):
    
    if len(PhraseUserEnd)==len(PhraseStim):
        list_nCorrectLetters = list()
        
        for element in range(0,len(PhraseStim)):
            sentence_phraseStim = PhraseStim[element][1]
            sentence_phraseUser = PhraseUserEnd[element][1]
            
            #print(sentence_phraseStim)
            #print(sentence_phraseUser)
            
            # edit sentence if it has a period at the end:
            #if sentence_phraseStim[-1] == '.':
            #    sentence_phraseStim = sentence_phraseStim[:-1]
            #if sentence_phraseUser[-1] != '.':
            #    sentence_phraseUser = sentence_phraseUser[:-1]
                
            # if pound sign in the sentence:
            # check first if it is in the phraseStim.
            if '£50' in sentence_phraseStim:
                # now if some form of currency (pound/kr/kr.) is typed, that will also be added to the phraseStim
                if 'pounds' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 pounds')
                    #print('new', sentence_phraseStim)
                elif 'pound' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 pound')
                    #print('new', sentence_phraseStim)
                
                elif '50kr.' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50kr.')
                    #print('new', sentence_phraseStim)
                elif '50kr' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50kr')
                    #print('new', sentence_phraseStim)
                
                elif '50 kr.' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 kr.')
                    #print('new', sentence_phraseStim)
                elif '50 kr' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 kr')
                    #print('new', sentence_phraseStim)
                
                elif 'L50' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', 'L50')
                    #print('new', sentence_phraseStim)
                
                else:
                    sentence_phraseStim = sentence_phraseStim.replace('£', '')
                    #print('new', sentence_phraseStim)
                    
            """
            # if people write [?] instead of the complete sentence
            if '[?]' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('[?]', '')
            
            # if people write .../..../..... instead of the complete sentence
            
            if '.....' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('.....', '')
            elif '....' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('....', '')
            elif '...' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('...', '')
            elif '..' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('..', '')
            """   
            
            print(sentence_phraseStim)
            print(sentence_phraseUser)
            print('\n')
            
            nCorrectLetters = max(len(sentence_phraseStim), len(sentence_phraseUser)) - distance.levenshtein(sentence_phraseStim.lower(), sentence_phraseUser.lower())
            #nCorrectLetters = len(sentence_phraseUser) - distance.levenshtein(sentence_phraseStim.lower(), sentence_phraseUser.lower())
            
            list_nCorrectLetters.append(nCorrectLetters)
            #print('Number of correct letters typed: ', nCorrectLetters, '   and number of letters to be typed: ', len(sentence_phraseStim))
            #print('\n')
            
    
    else:
        print('stimulation phrases and user phrases do not match')
        print('stimulation phrases:    ', PhraseStim)
        print('user phrases:    ', PhraseUserEnd)
        
    return list_nCorrectLetters

In [12]:
def EffectiveTimeFromUserKeys(UserKeys, PhrasesStim, PhraseUser, ValidityGazeLog, pathOfSession):
    
    UserKeysTime = [val[0] for val in UserKeys]
    ValidityGazeLog_time = [val[0] for val in ValidityGazeLog]
    
    # session name
    session_folder_name = pathOfSession.split('\\')[-1]
    
    timeTypingList = list()
    
    # time between and within fixations on a single letter
    ind_timePrevious = -1
    for time1 in UserKeysTime[1:]:
        ind_timePrevious = ind_timePrevious + 1
        time0 = UserKeysTime[ind_timePrevious]
        
        time, t1, t2 = time0.partition('+')
        timeStart = datetime.datetime.strptime(re.sub('[:.T]','-',time[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
        
        time, t1, t2 = time1.partition('+')
        timeEnd = datetime.datetime.strptime(re.sub('[:.T]','-',time[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
        timeBnPoints = timeEnd - timeStart
        
    
    # find starting point of every phrase 
    #print(len(PhrasesStim), len(PhraseUser))
    for ind in range(0,len(PhrasesStim)):
        #print(ind, PhrasesStim[ind][1], PhraseUser[ind][1])
        timePhraseStarts = PhrasesStim[ind][0]
        # Find the time when the phrase starts
        timePhraseStartsWUserKeys, ind_start = nearestTimePoint(UserKeysTime, timePhraseStarts)
        
        if '2019-01-16-15-42-51_2ndPart_1' in pathOfSession:
            if ind == 3:
                timePhraseStartsNew0 = timePhraseStartsWUserKeys + datetime.timedelta(seconds=19)
                seconds_start = timePhraseStartsWUserKeys[17:19]
                if int(seconds_start) > 41:
                    minute_start = PhraseUser[ind][0][14:16]
                    seconds_start_new = str(int(seconds_start) - 41) 
            
                    if int(minute_start) > 58:
                        minute_start_new = str(int(minute_start) - 59)
                        hour_start_new = str(int(timePhraseStartsWUserKeys[11:13]) + 1)
                        
                    else:
                        minute_start_new = str(int(minute_start) + 1)
                        hour_start_new = str(int(timePhraseStartsWUserKeys[11:13]))
                        
                else:
                    seconds_start_new = str(int(seconds_start) + 19)
                    minute_start_new = str(timePhraseStartsWUserKeys[14:16])
                    hour_start_new = str(int(timePhraseStartsWUserKeys[11:13]))
                    
                timePhraseStartsNew0 = timePhraseStartsWUserKeys[0:11] + hour_start_new + ':' + minute_start_new + ':' + seconds_start_new + timePhraseStartsWUserKeys[19:]
                #print('new start', timePhraseStartsNew)
                timePhraseStartsNew, ind_start = nearestTimePoint(UserKeysTime, timePhraseStartsNew0)
                #print(timePhraseStarts,UserKeys[ind_start])
                
        #print('start:', timePhraseStartsWUserKeys)
    
        seconds_end = PhraseUser[ind][0][17:19]
        
        #print('before', PhraseUser[ind][0])
        
        # for the actual end, add 5 seconds to the time
        if int(seconds_end) > 54:
            minute_end = PhraseUser[ind][0][14:16]
            seconds_end_new = str(int(seconds_end) - 55)
            
            if int(minute_end) > 58:
                minute_end_new = str(int(minute_end) - 59)
                hour_end_new = str(int(PhraseUser[ind][0][11:13]) + 1)
            else:
                minute_end_new = str(int(minute_end) + 1)
                hour_end_new = str(int(PhraseUser[ind][0][11:13]))
        
        else:
            seconds_end_new = str(int(seconds_end) + 5)
            minute_end_new = str(PhraseUser[ind][0][14:16])
            hour_end_new = str(int(PhraseUser[ind][0][11:13]))
        
        timePhraseEnds = PhraseUser[ind][0][0:11] + hour_end_new + ':' + minute_end_new + ':' + seconds_end_new + PhraseUser[ind][0][19:]
        
        #print('changed end', timePhraseEnds)
        
        ind_end_time, ind_end1 = nearestTimePoint(UserKeysTime, timePhraseEnds)
        #print(ind_start, ind_end1)
        #print(UserKeys[ind_end1])
        
        
        # find when the phrase ends:
        #if ind == len(PhrasesStim)-1: # if it is the last sentence, reverse index searching starts from the last index
        #    ind_end1 = len(UserKeys) - 1
        #else: # if it's not the last sentence, reverse index searching starts from the next phrase
            
            # find the time for when the sentence is completely written 
            
        #    ind_end_time, ind_end1 = nearestTimePoint(UserKeysTime, PhrasesStim[ind+1][0])
         
        #print('end: ', ind_end_time)
        
        #print('start to end', ind_start, ind_end1)
        
        ind_end2 = ind_end1
        ind_end_exists = False
        
        # reverse indexing to find 'NextPhrase' 
        while ind_end1 > ind_start+1:
            if UserKeys[ind_end1][1] == 'NextPhrase' and UserKeys[ind_end1][2]==1:
                ind_end = ind_end1
                ind_end_exists = True
            ind_end1 = ind_end1 - 1
            
        
        
        
        if not ind_end_exists:
            
            ind_end = ind_end2
        
        #print('ind end', ind_end)
        
        #print('end: ', UserKeys[ind_end][0])
        
        # reverse indexing to find 'NextPhrase' 
        #while ind_end1 > ind_start+1:
        #    ind_end1 = ind_end1 - 1
        #    if UserKeys[ind_end1][1] == 'NextPhrase' and UserKeys[ind_end1][2]==1:
        #        ind_end_start = ind_end1
        #    else ind_end_start = 0
        
        #if ind_end_start == 0:
        #    ind_end_start = ind_end 
        
        #while ind_end_start > ind_start + 1:
        #    ind_end_start = ind_end_start - 1
        #    if UserKeys[ind_end_start][1] == 'NextPhrase' and UserKeys[ind_end_start-1][1] != 'NextPhrase':
        #        ind_end = ind_end_start
        #        break
               
        
        flag_start = 0
        typing_list = list()
        letter_time_list = list()
        letter_list = list()
        keys_all_list = list()
        keys_ind_list = list()
        
        
        
        n_key = 0
        
        # collect the typing letters in a list of lists
        # leftshift do not count, because when comparing stim phrase and user phrase, the sentence is converted to lowercase
        for ind_typing in range(ind_start, ind_end):
            
            # create a list of indexes of added keys, to see later if the time between them should be counted or not
            
            if ind_typing > ind_start:
                if UserKeys[ind_typing-1][1] != UserKeys[ind_typing][1]:
                    n_key = n_key + 1
                    keys_all_list.append(UserKeys[ind_typing][1])
                    
                    
                    
                    
                    
            
            if len(UserKeys[ind_typing][1]) == 1 or UserKeys[ind_typing][1] in list_keysToBeCounted or 'Suggestion' in UserKeys[ind_typing][1]:
                
                #if not UserKeys[ind_typing][1].isdigit(): # time for writing digits need to be counted, since they will 
                # be counted in counting the correct letters
                 
                letter_time_list.append(UserKeys[ind_typing][0]) 
                #print(UserKeys[ind_typing][1])
                if UserKeys[ind_typing][1] != UserKeys[ind_typing+1][1]:
                        
                    # make a huge list of all things continuous to be added
                    #if len(UserKeys[ind_typing+1][1]) != 1 and UserKeys[ind_typing+1][1] not in list_keysToBeCounted and 'Suggestion' not in UserKeys[ind_typing+1][1]:
                        
                    #print(UserKeys[ind_typing][1], ',', UserKeys[ind_typing+1][1])
                    #print('letter time list', letter_time_list)
                    #print(UserKeys[ind_typing][1])
                    #print('')
                    typing_list.append(letter_time_list)
                    keys_ind_list.append(n_key)
                    letter_list.append(UserKeys[ind_typing][1])
                    letter_time_list = list()
                   
        
        #2019-01-16-15-42-print('start and end:', typing_list[0][0], typing_list[-1][-1])
        time1, t1, t2 = typing_list[0][0].partition('+')
        timeStart = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
        time2, t1, t2 = typing_list[-1][-1].partition('+')
        timeEnd = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
        timeDiff = timeEnd - timeStart
        #print(timeDiff.total_seconds())
        
        # add the time for typing every letter in a phrase
        time_typing_letter = 0
        ind_typing_letter = -1
        
        for letter_typed_list in typing_list:
            ind_typing_letter = ind_typing_letter + 1
            
            #if letter_list[ind_typing_letter] == '.' and ind_typing_letter < len(typing_list)-1: 
            #    if letter_list[ind_typing_letter+1] == '.':
            #        print('next one', letter_list[ind_typing_letter], letter_list[ind_typing_letter+1])
            #    if letter_list[ind_typing_letter-1] == '.':
            #        print('previous one', letter_list[ind_typing_letter-1], letter_list[ind_typing_letter])
            
            #print(letter_list[ind_typing_letter])
            
            time1, t1, t2 = letter_typed_list[0].partition('+')
            timeStart = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            time2, t1, t2 = letter_typed_list[-1].partition('+')
            timeEnd = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            timeDiff = timeEnd - timeStart
            
            time_typing_letter = time_typing_letter + timeDiff.total_seconds()
        
            
            # activation time is 250ms - add that for every key looked at, which will not be counted afterwards
            #time_typing_letter = time_typing_letter + 0.25
            
            
            
            #print(timeStart, timeEnd, timeDiff.total_seconds())
        
        timeTypingList.append(time_typing_letter)
        #print(time_typing_letter)

        
        #print(pathOfSession)
        
        
        #print('time in between')
        
        # add time between each letter, if it is valid to add --
        # for data more than 5s, not added at all
        # for data less than 5s, validity checked using gaze origin validity obtain from the tobii tracker, 
        # and if it shows valid pupil data for at least 50% of the time in between
        ind_letterTyped_previous = -1
        time_bnLetterCurrent = 0
        
        
        
        for letter_being_typed in typing_list[1:]:
            
            ind_letterTyped_previous = ind_letterTyped_previous + 1
            
            time_previousEnd_Str = typing_list[ind_letterTyped_previous][-1]
            time_currentStart_Str = letter_being_typed[0]
            
            time1, t1, t2 = time_previousEnd_Str.partition('+')
            time_previousEnd = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            time2, t1, t2 = time_currentStart_Str.partition('+')
            time_currentStart = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            time_bnLetters_notS = time_currentStart - time_previousEnd
            time_bnLetters = time_bnLetters_notS.total_seconds()
            
            
            if keys_ind_list[ind_letterTyped_previous+1] != keys_ind_list[ind_letterTyped_previous] + 1:
                timeTypingList[-1] = timeTypingList[-1] + 0.25 
                
                #print('activation time at', time_currentStart)
                continue
            
            #print(time_previousEnd, time_currentStart, time_bnLetters, 'between', letter_list[ind_letterTyped_previous], 'and', letter_list[ind_letterTyped_previous+1])
            
            if time_bnLetters < 5:
                
                if time_bnLetters < 2:
                    time_bnLetterCurrent = time_bnLetterCurrent + time_bnLetters
                    
                else:
                    #if '2019-01-16-17-00-12' in pathOfSession or '2019-01-17-15-31-12_2' in pathOfSession:
                    if session_folder_name in dict_noGazeData:
                        if ind == len(PhrasesStim)-1:
                            return timeTypingList
                        else:
                            continue
                    
                    # find the number of valid points in gaze
                    timeStart_gazeLog_Str, timeStart_gazeLog_ind = nearestTimePoint(ValidityGazeLog_time, time_previousEnd_Str)
                    timeEnd_gazeLog_Str, timeEnd_gazeLog_ind = nearestTimePoint(ValidityGazeLog_time, time_currentStart_Str)
                    validitySubset = [ValidityGazeLog[ind][1] for ind in range(timeStart_gazeLog_ind, timeEnd_gazeLog_ind)]
                    validity_pct = validitySubset.count(True)/len(validitySubset)
                    
                    if validity_pct > 0.8:
                        time_bnLetterCurrent = time_bnLetterCurrent + time_bnLetters
                    
        #print(timeTypingList[-1], time_bnLetterCurrent)    
        timeTypingList[-1] = timeTypingList[-1] + time_bnLetterCurrent
        #print(timeTypingList[-1])
        
    return timeTypingList

In [13]:
class DataForEveryTrial:
    subjectID = ''
    blockNumber = ''
    sessionNumber = ''
    variable = ''
    dataForTrial = ''
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        startrow = writer.sheets['Sheet1'].max_row
        dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
        
        writer.save()
     

In [15]:
metricComputed = 'EffectiveTime'
print(metricComputed)
dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\Subject_Block_Session_Trial_' + metricComputed +  '.xlsx'

            

j = 0
TimeDwellAvg = 0
#nSub = 0
TypingSpeed = list()
Names = list()

list_typingSpeed_trial = list()
list_typingSpeed_subject = list()

for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        #if 'ls2\\4\\2019-02-18-10-28-35_2' not in root:
        #    continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        
        
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if scratchPad is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            
            # find the language of the data:
            if 'Hvad er det fulde navn på dit universitet?' in phraseLog[1][1]: 
                language = 'Danish'
            else:
                language = 'English'
                
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
                
            # find dwell time of typing
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new, root)
                
            phraseStim_reduced = stimPhrasesEdit(phraseLog_new, root)
            
            # if it is the 2nd part of the session, picture is not described:
            if '2ndPart' in root:
                picture = 'not_described'
            else:
                picture = 'described'
            
            phraseUserEnd_reduced = scratchPadPhraseEdit(scratchPad_new, subjName, root, picture)
            
            
            # find validity of gaze log -- combine a validity measure with an or of right and left gaze
            time_gazeLog = [item[0] for item in gazeLog]
            validity_leftGaze = [True if item[2]=='Valid' else False for item in gazeLog ]
            validity_rightGaze = [True if item[9] == 'Valid' else False for item in gazeLog ]
            
            validity_gazeLog = [[time_gazeLog[i], validity_leftGaze[i] or validity_rightGaze[i]] for i in range(0, len(time_gazeLog)) ]
            
            #print('stim phrases', phraseStim_reduced)
            #print('user phrases', phraseUserEnd_reduced)
            
            if '1stPart' in root:
                print('1stPart')
                phraseUserEnd_reduced1 = phraseUserEnd_reduced
                phraseStim_reduced1 = phraseStim_reduced
                userKeys_wDwellTime1 = userKeys_wDwellTime
                validity_gazeLog1 = gazeLog
                continue
            
            
            
            if '2ndPart' in root:
                print('2ndPart')
                phraseUserEnd_reduced2 = phraseUserEnd_reduced
                phraseStim_reduced2 = phraseStim_reduced
                userKeys_wDwellTime2 = userKeys_wDwellTime
                validity_gazeLog2 = validity_gazeLog
                
                
                
                phraseUserEnd_reduced = phraseUserEnd_reduced1 + phraseUserEnd_reduced2
                phraseStim_reduced = phraseStim_reduced1 + phraseStim_reduced2
                userKeys_wDwellTime = userKeys_wDwellTime1 + userKeys_wDwellTime2
                validity_gazeLog = validity_gazeLog1 + validity_gazeLog2
                
                phraseUserEnd_reduced1 = list()
                phraseStim_reduced1 = list()
                userKeys_wDwellTime1 = list()
                validity_gazeLog1 = list()
                
            #print(len(phraseUserEnd_reduced), len(phraseStim_reduced))
                
            # check if the number of sentences match in the input and the output
            if len(phraseUserEnd_reduced)!=len(phraseStim_reduced):
                print('needs fixing of number of phrases')
                #print('before', phraseStim_reduced)
                #phraseStim_toMatch, phraseUserEnd_toMatch = MatchPhraseNumbers(phraseStim_reduced, phraseUserEnd_reduced)
                #print('after', phraseStim_toMatch)
            else:
                phraseStim_toMatch = phraseStim_reduced
                phraseUserEnd_toMatch = phraseUserEnd_reduced
                
            
            # Effective time of typing:
            # Find time of typing only every letter, numbers, space and changing of keyboards
            effective_time = EffectiveTimeFromUserKeys(userKeys_wDwellTime, phraseStim_toMatch, phraseUserEnd_toMatch, validity_gazeLog, root)
            
            
            dataToSave = DataForEveryTrial()
            dataToSave.subjectID = subjAndSessionName.split('__')[0]
            dataToSave.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave.sessionNumber = subjAndSessionName[-1]
            dataToSave.variable = metricComputed
            dataToSave.dataForTrial = effective_time
            dataToSave.resultPathName = resultFileName
            
            print(dataToSave.printInfo())
            dataToSave.AddToFile()
            
            effective_time = list()
        

EffectiveTime
subject and session name:  ac__1__2019-02-11-11-18-30_1
  subjectID block session  trial  EffectiveTime
0        ac     1       1      0      88.519338
1        ac     1       1      1      64.398373
2        ac     1       1      2      29.620000
3        ac     1       1      3      90.961113
4        ac     1       1      4      35.191650
subject and session name:  ac__1__2019-02-11-11-33-32_2
  subjectID block session  trial  EffectiveTime
0        ac     1       2      0     142.750796
1        ac     1       2      1     242.865031
2        ac     1       2      2     194.700100
3        ac     1       2      3     229.921825
4        ac     1       2      4     203.791808
subject and session name:  ac__2__2019-02-12-15-00-4_1
  subjectID block session  trial  EffectiveTime
0        ac     2       1      0     289.365584
1        ac     2       1      1      73.270160
2        ac     2       1      2     152.691225
3        ac     2       1      3     157.256666
4  

subject and session name:  aq__2__2019-02-07-11-24-49_2
  subjectID block session  trial  EffectiveTime
0        aq     2       2      0     159.151574
1        aq     2       2      1      49.405609
2        aq     2       2      2     265.236820
3        aq     2       2      3     187.572785
4        aq     2       2      4     303.118169
subject and session name:  aq__3__2019-02-08-11-33-53_1stPart_1
1stPart
subject and session name:  aq__3__2019-02-08-12-11-34_2ndPart_1
2ndPart
  subjectID block session  trial  EffectiveTime
0        aq     3       1      0     358.569867
1        aq     3       1      1     232.188920
2        aq     3       1      2     271.322369
3        aq     3       1      3     143.700484
4        aq     3       1      4     258.961662
subject and session name:  aq__3__2019-02-08-12-22-35_2
  subjectID block session  trial  EffectiveTime
0        aq     3       2      0      75.428323
1        aq     3       2      1      60.943334
2        aq     3       

  subjectID block session  trial  EffectiveTime
0       bh2     3       2      0     170.903908
1       bh2     3       2      1     106.634341
2       bh2     3       2      2     100.701105
3       bh2     3       2      3      87.305040
4       bh2     3       2      4      67.371328
subject and session name:  bh2__4__2019-03-04-10-14-44_1
  subjectID block session  trial  EffectiveTime
0       bh2     4       1      0     173.616921
1       bh2     4       1      1     135.608200
2       bh2     4       1      2     128.192759
3       bh2     4       1      3      82.381091
4       bh2     4       1      4      86.835630
subject and session name:  bh2__4__2019-03-04-10-38-49_2
  subjectID block session  trial  EffectiveTime
0       bh2     4       2      0      34.087772
1       bh2     4       2      1      31.296979
2       bh2     4       2      2      36.714204
3       bh2     4       2      3      67.528387
4       bh2     4       2      4      42.241011
subject and session na

subject and session name:  jm__5__2019-02-06-16-10-7_2
  subjectID block session  trial  EffectiveTime
0        jm     5       2      0      31.197389
1        jm     5       2      1      44.865796
2        jm     5       2      2      30.056205
3        jm     5       2      3      74.465826
4        jm     5       2      4      63.930894
subject and session name:  jp__1__2019-02-11-14-23-27_1
  subjectID block session  trial  EffectiveTime
0        jp     1       1      0     159.394476
1        jp     1       1      1      72.193466
2        jp     1       1      2      83.626422
3        jp     1       1      3     117.686075
4        jp     1       1      4      57.011547
subject and session name:  jp__1__2019-02-11-14-46-36_2
  subjectID block session  trial  EffectiveTime
0        jp     1       2      0      95.206294
1        jp     1       2      1     344.924067
2        jp     1       2      2     215.030381
3        jp     1       2      3     247.832607
4        jp     1

subject and session name:  le__2__2019-02-19-10-22-45_2
  subjectID block session  trial  EffectiveTime
0        le     2       2      0     212.257313
1        le     2       2      1     270.828536
2        le     2       2      2     207.180622
3        le     2       2      3     253.374425
4        le     2       2      4      56.144226
subject and session name:  le__3__2019-02-21-15-01-4_1stPart_1
1stPart
subject and session name:  le__3__2019-02-21-15-25-56_2ndPart_1
2ndPart
  subjectID block session  trial  EffectiveTime
0        le     3       1      0     191.769870
1        le     3       1      1     107.647809
2        le     3       1      2      74.029826
3        le     3       1      3     139.272377
4        le     3       1      4     302.657839
subject and session name:  le__3__2019-02-21-15-36-13_2
  subjectID block session  trial  EffectiveTime
0        le     3       2      0     109.756627
1        le     3       2      1      51.565393
2        le     3       2

subject and session name:  ls2__4__2019-02-18-10-28-35_2
  subjectID block session  trial  EffectiveTime
0       ls2     4       2      0     159.464864
1       ls2     4       2      1      79.360609
2       ls2     4       2      2     115.317166
3       ls2     4       2      3     128.656048
4       ls2     4       2      4     212.636387
subject and session name:  ls2__5__2019-02-19-13-08-35_1
  subjectID block session  trial  EffectiveTime
0       ls2     5       1      0     168.114204
1       ls2     5       1      1     179.645759
2       ls2     5       1      2     132.019420
3       ls2     5       1      3      73.354856
4       ls2     5       1      4     113.587508
subject and session name:  ls2__5__2019-02-19-13-34-31_2
  subjectID block session  trial  EffectiveTime
0       ls2     5       2      0      43.351962
1       ls2     5       2      1     143.817507
2       ls2     5       2      2      53.225163
3       ls2     5       2      3      59.820688
4       ls2  

  subjectID block session  trial  EffectiveTime
0        no     1       1      0     127.660945
1        no     1       1      1     136.264881
2        no     1       1      2     212.520804
3        no     1       1      3     133.500361
subject and session name:  no__1__2019-01-16-15-51-13_2
  subjectID block session  trial  EffectiveTime
0        no     1       2      0      88.595330
1        no     1       2      1      78.473092
2        no     1       2      2      81.205318
3        no     1       2      3      97.641668
4        no     1       2      4      76.320263
subject and session name:  no__2__2019-01-28-10-01-27_1
  subjectID block session  trial  EffectiveTime
0        no     2       1      0      47.990255
1        no     2       1      1      42.643289
2        no     2       1      2      77.934867
3        no     2       1      3      71.470968
4        no     2       1      4      80.183319
subject and session name:  no__2__2019-01-28-10-17-12_2
  subjectID bloc

  subjectID block session  trial  EffectiveTime
0        rh  3_MS       1      0     104.231850
1        rh  3_MS       1      1     110.909291
2        rh  3_MS       1      2      31.731662
3        rh  3_MS       1      3     198.331225
4        rh  3_MS       1      4     158.283539
subject and session name:  rh__3_MS__2019-03-14-13-56-56_2
  subjectID block session  trial  EffectiveTime
0        rh  3_MS       2      0      84.225010
1        rh  3_MS       2      1     232.194972
2        rh  3_MS       2      2      79.969689
3        rh  3_MS       2      3     145.120665
subject and session name:  rh__4__2019-04-11-13-03-50_1
  subjectID block session  trial  EffectiveTime
0        rh     4       1      0     177.380828
1        rh     4       1      1     167.788238
2        rh     4       1      2     241.059198
3        rh     4       1      3     237.570836
4        rh     4       1      4     211.850286
subject and session name:  rh__4__2019-04-11-13-40-31_2
  subjectID b

In [14]:
# check using character levenshtein distance
                dissimilarity_check_2ndLastStimPhrase = distance.levenshtein(phraseLog_reduced[-2][1].lower(), phraseUserEnd_reduced[-1][1].lower())
                dissimilarity_check_LastStimPhrase = distance.levenshtein(phraseLog_reduced[-1][1].lower(), phraseUserEnd_reduced[-1][1].lower())
                
                print(dissimilarity_check_LastStimPhrase, dissimilarity_check_2ndLastStimPhrase)
                if dissimilarity_check_LastStimPhrase > dissimilarity_check_2ndLastStimPhrase:
                    del phraseLog_reduced[-1]

IndentationError: unexpected indent (<ipython-input-14-c2326d0dca13>, line 2)

In [ ]:
def phrasesTyped(your_list, timeList, userKeys, stimPhraseList):
    # Find entries in scratchpad (end of each trial)
    
    IndexNotInclude = list()
    PhraseList = list()
    TypingSpeed = list()

    # After every complete phrase, the next phrase is empyty ''
    for row_ind in range(0, len(your_list)):
        if your_list[row_ind][1] == '':
            if row_ind == 1:# first empty phrase needs to be not considered , at least for most of the subjects
                continue
            if your_list[row_ind][1] == your_list[row_ind-1][1]:
                continue
            IndexNotInclude.append(row_ind)
    
    # Find the time taken for each phrase  

    #print(len(timeList), len(stimPhraseList), len(IndexNotInclude))
    #print(stimPhraseList)
    if len(IndexNotInclude) != len(timeList) :
        #print(IndexNotInclude, timeList)
        print('error and both are not same')
        #print(IndexNotInclude)
        #print(scratchPad)
    else:
        
        for indPhrase in range(0, len(IndexNotInclude)):
            startPhrase = timeList[indPhrase]    
            endPhrase = IndexNotInclude[indPhrase]-1
            headStartPhrase,sep,tail = userKeys[startPhrase][0].partition('+')
            headEndPhrase,sep,tail = your_list[endPhrase][0].partition('+')
            timeTaken = datetime.datetime.strptime(re.sub('[:.T]','-',headEndPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]', '-', headStartPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            phraseTyped = your_list[endPhrase][1]
            PhraseList.append([timeTaken, phraseTyped])
            #print(startPhrase, ',', endPhrase, ',', headStartPhrase, ',', headEndPhrase, ',', timeTaken, ',', phraseTyped)
            
        
    #print(timeTaken)
#    print(PhraseList)
    
    # Compute the total time for the phrase and the total number of words typed during that time (1 word = 5 characters, including space)
    timeAdd = datetime.datetime.strptime('1000-1-1-0-00-00-000000',"%Y-%m-%d-%H-%M-%S-%f")
    characterLen = 0
    
    for indPhrase in range(0, len(IndexNotInclude)):
        timeAdd += PhraseList[indPhrase][0]
        characterLen += len(PhraseList[indPhrase][1])
#        print(timeAdd, ': ', characterLen)
#    print(timeAdd, ': ', characterLen)
        
    TimeTaken = (timeAdd.microsecond/1000000 + timeAdd.second)/60 + timeAdd.minute
#    print(TimeTaken)
    WPM = (characterLen)/(5*TimeTaken)
    print("Typing speed is", WPM)
    
    return WPM

In [ ]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201805_HealthnRehab\data_summary.csv'

    if Path(fileIn).is_file():
        print(fileIn)
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[0])
                                
                                if rowJoined[1] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[1])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[1] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[1])])])
            
            
            else:
                fileOut = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                filewriter.writerow([subjName[0], DataEvaluated[ind[0]]])
    

In [ ]:
def phrasesTyped2(your_list):
    # Find entries in scratchpad that are the start and end of the phrase
    
    IndexNotInclude = list()
    
    # After every complete phrase, the next phrase is empyty '', and the first phrase is 'scratchPadText'
    for row_ind in range(0, len(your_list)):
        if your_list[row_ind][1] in NotInclude:
            if row_ind == 0:
                IndexNotInclude.append(row_ind) # first element is to be considered for the first phrase start time
                continue
            if your_list[row_ind][1] == your_list[row_ind-1][1]:
                continue
            IndexNotInclude.append(row_ind)
    
    for row_ind in range(0, len(your_list)):
        list_inner = your_list[row_ind]
        if list_inner[1] in NotInclude:
            IndexNotInclude.append(row_ind)
    
    # Remove consecutive entries with blank scratchpads

    IndexNotIncludeArray = np.asarray(IndexNotInclude)
    IndDiff = np.diff(IndexNotIncludeArray,1)
    
    ind = len(IndexNotInclude)-1
    while (ind>-1):
        if IndDiff[ind] == 1:
            for i_2 in range (ind,len(IndexNotInclude)):
                IndexNotInclude[i_2] = IndexNotInclude[i_2]-1
        
            del(IndexNotInclude[ind])
            del your_list[IndexNotIncludeArray[ind]]
        ind = ind - 1
        
        # Find the time taken for each phrase
    PhraseList = list()

    for indPhrase in range(0, len(IndexNotInclude)-1):
        startPhrase = IndexNotInclude[indPhrase]
        endPhrase = IndexNotInclude[indPhrase+1]
        headStartPhrase,sep,tail = your_list[startPhrase+1][0].partition('+')
        headEndPhrase,sep,tail = your_list[endPhrase-1][0].partition('+')
        print(headStartPhrase, headEndPhrase)
        timeTaken = datetime.datetime.strptime(re.sub('[:.T]','-',headEndPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]', '-', headStartPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
        phraseTyped = your_list[endPhrase-1][1]
        PhraseList.append([timeTaken, phraseTyped])
        
    #print(timeTaken)
    #print(PhraseList)
    
    # Compute the total time for the phrase and the total number of words typed during that time (1 word = 5 characters, including space)
    timeAdd = datetime.datetime.strptime('1000-1-1-0-00-00-000000',"%Y-%m-%d-%H-%M-%S-%f")
    characterLen = 0

    for indPhrase in range(0, len(IndexNotInclude)-1):
        timeAdd += PhraseList[indPhrase][0]
        characterLen += len(PhraseList[indPhrase][1])
    
    TimeTaken = (timeAdd.microsecond/1000000 + timeAdd.second)/60 + timeAdd.minute
    WPM = (characterLen)/(5*TimeTaken)
    #print("Typing speed is", WPM)

In [ ]:
def MatchPhraseNumbers(phraseStim_reduced, phraseUserEnd_reduced):
    
    print(len(phraseUserEnd_reduced), len(phraseStim_reduced))
    
    # check if the last stim phrase is not typed
                
    # check using character levenshtein distance
    dissimilarity_check_2ndLastStimPhrase = distance.levenshtein(phraseStim_reduced[-2][1].lower(), phraseUserEnd_reduced[-1][1].lower())
    dissimilarity_check_LastStimPhrase = distance.levenshtein(phraseStim_reduced[-1][1].lower(), phraseUserEnd_reduced[-1][1].lower())
                
    #print(dissimilarity_check_LastStimPhrase, dissimilarity_check_2ndLastStimPhrase)
    if dissimilarity_check_LastStimPhrase > dissimilarity_check_2ndLastStimPhrase:
        del phraseStim_reduced[-1]
                    
    # check if the last stim phrase is not typed
                
    # check using character levenshtein distance
    dissimilarity_check_2ndLastStimPhrase = distance.levenshtein(phraseStim_reduced[-2][1].lower(), phraseUserEnd_reduced[-1][1].lower())
    dissimilarity_check_LastStimPhrase = distance.levenshtein(phraseStim_reduced[-1][1].lower(), phraseUserEnd_reduced[-1][1].lower())
                
    #print(dissimilarity_check_LastStimPhrase, dissimilarity_check_2ndLastStimPhrase)
    if dissimilarity_check_LastStimPhrase > dissimilarity_check_2ndLastStimPhrase:
        del phraseStim_reduced[-1]
                    
    if len(phraseUserEnd_reduced)==len(phraseStim_reduced):
        #print(len(phraseUserEnd_reduced))
        return phraseStim_reduced, phraseUserEnd_reduced
                    
    # check using word levenshtein distance
    # initialize the stemmer
    if language == 'Danish':
        words_stemmer = snowball.DanishStemmer()
    else:
        words_stemmer = snowball.EnglishStemmer()
                    
    roots_phraseStim = list()
    roots_phraseUser = list()
                    
    for word in phraseStim_reduced[-2][1].split():
        # find roots of words in stimulation phrase - 
        roots_phraseStim.append(words_stemmer.stem(word))

    for word in phraseUserEnd_reduced[-1][1].split():
        roots_phraseUser.append(words_stemmer.stem(word))
                
    dissimilar_words_2ndLastPhraseStim = distance.levenshtein(roots_phraseStim, roots_phraseUser)
                                
    for word in phraseStim_reduced[-1][1].split():
        # find roots of words in stimulation phrase - 
        roots_phraseStim.append(words_stemmer.stem(word))

    for word in phraseUserEnd_reduced[-1][1].split():
        roots_phraseUser.append(words_stemmer.stem(word))
                
    dissimilar_words_LastPhraseStim = distance.levenshtein(roots_phraseStim, roots_phraseUser)
                
    if dissimilar_words_2ndLastPhraseStim < dissimilar_words_LastPhraseStim:
        del phraseStim_reduced[-1]
    
    
    
    return phraseStim_reduced, phraseUserEnd_reduced


In [ ]:
def AddToFile(typing_speed, subjectID, resultPathName):
   
    
    
    wb = load_workbook(resultPathName)
    ws = wb.worksheets[0]
    
    typing_speed.insert(0, subjectID)
    ws.append(typing_speed)
    
    
    
    wb.save(resultPathName)